In [121]:
import pandas as pd

In [122]:
df = pd.read_csv("dpschool2a.csv", delimiter=";")

In [123]:
df = df[df["published"] == 1]

In [124]:
df = df[['title', 'short_description', 'description', 'latitude', 'longitude', 'address', 'addr_no', 
         'zip_code', 'city', 'country', 'additional_fields', 'image', 'icons', 'modified']]

In [125]:
import json
df["additional_fields"] = df["additional_fields"].map(lambda x: json.loads(x))
df["image"] = df["image"].map(lambda x: json.loads(x))

In [126]:
df = df[df['latitude'].notna()]

In [127]:
from pandas import isnull
df["icons"] = df["icons"].map(lambda x: {} if isnull(x) else x)
df["icons"] = df["icons"].map(lambda x: x if isinstance(x, str) else str(x))
df["icons"] = df["icons"].map(lambda x: json.loads(x))

In [128]:
df["modified"] = pd.to_datetime(df["modified"], errors="coerce")

In [129]:
df['longitude'] = df['longitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000
df['latitude'] = df['latitude'].map(lambda x: x.replace('.', "")[:6]).astype(float) / 10000

In [130]:
df["description"] = df["description"].map(lambda x: "" if isnull(x) else x)
df["short_description"] = df["short_description"].map(lambda x: "" if isnull(x) else x)
df["addr_no"] = df["addr_no"].map(lambda x: "" if isnull(x) else x)

In [131]:
df["address"] = df["address"] + " " + df["addr_no"]

In [132]:
df.drop(["addr_no"], axis=1, inplace=True)

In [133]:
df["position"] = None
for i in df.index:
    df.at[i, 'position'] = (df.loc[i]['latitude'], df.loc[i]['longitude'])

In [134]:
icons = {"Toilette für Alle": 10, "Rolli WC": 11, "Rolli WC -DIN fern": 20, "WC eng": 17, "kein Rolli WC vorhanden": 21,
        "Parken": 7, "Stufe": 1, "WC - unisex": 4, "WC getrennt": 6, "Rollstuhl": 5, "Lift groß": 12, "Lift klein": 13,
        "Kino-Untertitel": 15, "Induktionsschleife": 14, "low-vision": 18, "kognitiv beeinträchtigt": 19, "Umkleide": 9,
        "Dusche": 8, "nicht rollstuhlgerecht": 16, "Rampe": 2, "keine Steigung - eben": 26, "moderate Steigung / Gefälle": 22,
        "weitere moderate Steigungen / Gefälle": 23, "heftige Steigung / Gefälle": 25, "weitere heftige Steigung / Gefälle": 24}
new_dict = {v: k for k, v in icons.items()}
new_dict.update({40: "Euro Key", 41: "Ramp Incline", 42: "Door Width"})

In [135]:
df["icons"] = df["icons"].map(lambda x: tuple(map(int, list(x.values()))))

In [136]:
df[40] = df["additional_fields"].map(lambda x: x["wc_euro"] if "wc_euro" in x.keys() else "0")
df[40] = df[40].map(lambda x: int(x) if x in ["1", "0"] else 0)

In [137]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

df[41] = df["additional_fields"].map(lambda x: x["wc_ramp_incline"])
df[41] = df[41].map(lambda x: x.replace(",","."))
df[41] = df[41].map(lambda x: float(x) if isfloat(x) else 0)
#need to check further as some have extra text

In [138]:
df[42] = df["additional_fields"].map(lambda x: x["wc_door_width"])
df[42] = df[42].map(lambda x: int(x) if x != "" else 150)
df[42] = df[42].map(lambda x: x if x > 15 else x*10)

In [139]:
x = pd.get_dummies(df.icons.apply(pd.Series).stack()).sum(level=0)
x = x.rename(int, axis="columns")
col = list(x.columns)
for i in [40, 41, 42]:
    col.append(i)

In [140]:
df = df.rename(columns={40:"eurokey", 41: "ramp_steepness", 42: "door_width"})

In [141]:
df["photo"] = df["image"].map(lambda x: x["image_small"])
df["plan"] = df["additional_fields"].map(lambda x: "images/wc-plan/" + x["wc_plan"] + ".jpg")
#df["seat_height"] = df["additional_fields"].map(lambda x: x["wc_seat"])

In [142]:
df = df.drop(df.loc[df['icons'].isin([(21,)])].index)

In [143]:
df["grip"] = df["additional_fields"].map(lambda x:x["wc_wheregrip"])

In [104]:
df.head()

,title,short_description,description,latitude,longitude,address,zip_code,city,country,additional_fields,image,icons,modified,position,eurokey,ramp_steepness,door_width,photo,plan,grip
1,WC Bad Georgenschwaige Damen,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Damen-WC</strong> ist im Umklei...,48.1756,11.5717,Belgradstr. 195,80804,MÃ¼nchen,DE,"{'wc_shortname': 'ba-george1WC1', 'wc_plan': '...",{'image_small': 'images/wc-af/ba-george1WC1f1....,"(4, 11)",2021-09-08 09:21:00,"(48.1756, 11.5717)",0,0.0,85,images/wc-af/ba-george1WC1f1.jpg,images/wc-plan/ba-george1WC1.jpg,lr
2,WC Bad Georgenschwaige Herren,Bedingt Ã¶ffentliche Toilette in einem Freibad.,<p>Das <strong>Herren-WC ist im Umkleidebau</s...,48.1756,11.5717,Belgradstr. 195,80804,MÃ¼nchen,DE,"{'wc_shortname': 'ba-george1WC2', 'wc_plan': '...",{'image_small': 'images/wc-af/ba-george1WC2f1....,"(6, 20)",2021-09-08 09:21:00,"(48.1756, 11.5717)",0,0.0,85,images/wc-af/ba-george1WC2f1.jpg,images/wc-plan/ba-george1WC2.jpg,l
3,WC Freibad Unterhaching,Bedingt Ã¶ffentliche Toilette mit Dusche in ei...,<p>Das rollstuhlgerechte WC des Unterhachinger...,48.0619,11.6208,Schrenkstr. 2,82008,Unterhaching,Deutschland,"{'wc_shortname': 'ba-unterh1WC', 'wc_plan': 'b...",{'image_small': 'images/wc-af/ba-unterh1WC1f1....,"(4, 8, 11)",2021-10-06 08:10:00,"(48.0619, 11.6208)",1,0.0,75,images/wc-af/ba-unterh1WC1f1.jpg,images/wc-plan/ba-unterh1WC.jpg,lr
4,WC U-Bahnhof MÃ¼nchner Freiheit,"Ã–ffentliche Toilette im U-Bahnhof, zugÃ¤nglic...",<p>Das rollstuhlgerechte WC des U-Bahnhofs MÃ¼...,48.1622,11.5870,MÃ¼nchner Freiheit,80802,MÃ¼nchen,DE,"{'wc_shortname': 'bf-muefre1WC', 'wc_plan': 'b...",{'image_small': 'images/wc-af/bf-muefre1WC1f1....,"(4, 11, 12)",2021-10-06 08:09:00,"(48.1622, 11.587)",1,0.0,85,images/wc-af/bf-muefre1WC1f1.jpg,images/wc-plan/bf-muefre1WC1.jpg,lr
5,WC Dynasty Unterhaching,HalbÃ¶ffentliche Toilette in einem Lokal.,<p>Auf dem Weg zum Haupteingang ist eine Schwe...,48.0556,11.6076,Inselkammerstr. 3,82008,Unterhaching,Deutschland,"{'wc_shortname': 'dynast1', 'wc_plan': 'dynast...",{'image_small': 'images/wc-af/dynast1WC1f1.jpg...,"(4, 17)",2021-08-18 16:24:00,"(48.0556, 11.6076)",0,0.0,90,images/wc-af/dynast1WC1f1.jpg,images/wc-plan/dynast1WC.jpg,rfl


lr = flexible left & flexible right

l = flexible left

rfl & flr = fixed left and flexible right

fr = fixed right

none = no grip

lfr = flexible left and fixed right

r = flexible right

fl = fixed left

In [144]:
df["access"] = df["additional_fields"].map(lambda x:x["wc_infos"])
df["interior_description"] = df["additional_fields"].map(lambda x:x["wc_roominfos"])
df["wc_details"] = df["additional_fields"].map(lambda x:x["wc_addinfo"])
df["wc_accessleft"] = df["additional_fields"].map(lambda x:x["wc_accessleft"])
df["wc_accessright"] = df["additional_fields"].map(lambda x:x["wc_accessright"])

In [145]:
df[df["title"] == "WC Paulaner am Nockherberg - Biergarten"]["wc_details"][80]

'<p>Das WC-Becken ist links von der Tür. Die WC Spülung ist am rechten Bügel und hinter dem WC-Becken in 110 cm Höhe möglich.<br />Aus der Sitzposition gesehen hängt rechts eine Notrufschnur bis 30 cm über dem Boden.</p>'

In [146]:
df = df.rename(columns={"description": "direction"})

In [148]:
df = df[["title", "short_description", "photo", "plan", "position", "address", "eurokey", "ramp_steepness", "door_width", "modified",
         "zip_code", "city", "grip", "direction", "access", "interior_description", "wc_details", "wc_accessright", "wc_accessleft"]]

In [149]:
errors = pd.read_excel("toilets_text errors.xlsx")

In [150]:
errors.dropna(inplace=True)

In [151]:
list(errors["Text error"])

['Ãœ', 'Ã¼', 'Ã–', 'Ã¶', 'Ã„', 'Ã¤', 'ÃŸ', 'Ã©', 'Ã¨', 'Ã‡', 'â€“']

In [152]:
import re

errors = pd.Series(errors["correction"].values,index=errors["Text error"]).to_dict()

# use these three lines to do the replacement
rep = dict((re.escape(k), v) for k, v in errors.items()) 
#Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
pattern = re.compile("|".join(rep.keys()))

In [153]:
df["short_description"] = df["short_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["title"] = df["title"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["address"] = df["address"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["city"] = df["city"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["direction"] = df["direction"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["access"] = df["access"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["interior_description"] = df["interior_description"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))
df["wc_details"] = df["wc_details"].map(lambda x: pattern.sub(lambda m: rep[re.escape(m.group(0))], x))

In [154]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [155]:
df["direction"] = df["direction"].map(lambda x:remove_html_tags(x))
df["access"] = df["access"].map(lambda x:remove_html_tags(x))
df["interior_description"] = df["interior_description"].map(lambda x:remove_html_tags(x))
df["wc_details"] = df["wc_details"].map(lambda x:remove_html_tags(x))

In [156]:
df["id"] = df.index

In [157]:
df.head()

,title,short_description,photo,plan,position,address,eurokey,ramp_steepness,door_width,modified,zip_code,city,grip,direction,access,interior_description,wc_details,wc_accessright,wc_accessleft,id
1,WC Bad Georgenschwaige Damen,Bedingt öffentliche Toilette in einem Freibad.,images/wc-af/ba-george1WC1f1.jpg,images/wc-plan/ba-george1WC1.jpg,"(48.1756, 11.5717)",Belgradstr. 195,0,0.0,85,2021-09-08 09:21:00,80804,München,lr,"Das Damen-WC ist im Umkleidebau, ganz hinten l...",,"Das Waschbecken ist außerhalb des WC-Raums, di...",Seitlich hinter dem WC ist auf 100 cm ein Tast...,93,85,1
2,WC Bad Georgenschwaige Herren,Bedingt öffentliche Toilette in einem Freibad.,images/wc-af/ba-george1WC2f1.jpg,images/wc-plan/ba-george1WC2.jpg,"(48.1756, 11.5717)",Belgradstr. 195,0,0.0,85,2021-09-08 09:21:00,80804,München,l,"Das Herren-WC ist im Umkleidebau, ganz hinten ...",,"Das Waschbecken ist außerhalb des WC-Raums, di...",Rechts an der Wand ist in 80 cm Höhe die Spülu...,,90,2
3,WC Freibad Unterhaching,Bedingt öffentliche Toilette mit Dusche in ein...,images/wc-af/ba-unterh1WC1f1.jpg,images/wc-plan/ba-unterh1WC.jpg,"(48.0619, 11.6208)",Schrenkstr. 2,1,0.0,75,2021-10-06 08:10:00,82008,Unterhaching,lr,Das rollstuhlgerechte WC des Unterhachinger Fr...,Das WC-Becken ist schräg links gegenüber von d...,Zwischen Umkleide und Duschbad ist eine Schieb...,,105,75,3
4,WC U-Bahnhof Münchner Freiheit,"Öffentliche Toilette im U-Bahnhof, zugänglich ...",images/wc-af/bf-muefre1WC1f1.jpg,images/wc-plan/bf-muefre1WC1.jpg,"(48.1622, 11.587)",Münchner Freiheit,1,0.0,85,2021-10-06 08:09:00,80802,München,lr,Das rollstuhlgerechte WC des U-Bahnhofs Münchn...,,Der Bügel an der Türinnenseite ist eigentlich ...,Aus der Sitzposition gesehen hängt links vom W...,100,105,4
5,WC Dynasty Unterhaching,Halböffentliche Toilette in einem Lokal.,images/wc-af/dynast1WC1f1.jpg,images/wc-plan/dynast1WC.jpg,"(48.0556, 11.6076)",Inselkammerstr. 3,0,0.0,90,2021-08-18 16:24:00,82008,Unterhaching,rfl,Auf dem Weg zum Haupteingang ist eine Schwelle...,Das WC befindet sich im Erdgeschoss des Lokals...,,,104,,5


In [158]:
df.drop([315, 1091, 20, 1090, 1088, 1086, 1087, 1089], inplace=True)
df.to_json("toilets_v13.json")